In [1]:
import numpy as np

In [2]:
def f_padd(I,p):
    numRows = I.shape[0]
    numCols = I.shape[1]
    zeroRows = np.zeros((p,numCols))
    I = np.vstack((zeroRows,I))
    I = np.vstack((I,zeroRows))
    zeroCols = np.zeros((numRows+2*p,p))
    I = np.hstack((zeroCols,I))
    I = np.hstack((I,zeroCols))
    return I

In [3]:
def f_conv2d(I,K,p):
    fSize = K.shape[0]
    I2 = f_padd(I,p)
    numRows = I2.shape[0]
    numCols = I2.shape[1]
    C = np.zeros((numRows-2*p,numCols-2*p))
    for i in range(numRows-fSize+1):
        for j in range(numCols-fSize+1):
            A = I2[i:i+fSize,j:j+fSize]
            C[i,j] = (A.flatten()*K.flatten()).sum()
    return C
    

In [4]:
def f_ReLU(C):
    C[C<0] = 0
    return C

In [5]:
def f_pool(C):
    r = C.shape[0]
    c = C.shape[1]
    S = np.zeros((int(r/2),int(c/2)))
    for i in range(0,r,2):
        for j in range(0,c,2):
            S[int(i/2),int(j/2)] = C[i:i+2,j:j+2].max()
    return S

In [6]:
def f_sigmoid(f,w,bf):
    x = w.dot(f)+bf
    y_hat = 1/(1+np.exp(-x))
    return y_hat

In [7]:
def f_forwardPass(I,K,b,w,bf):
    p = int(K.shape[0]/2)
    C = f_conv2d(I,K,p)
    C = C+b
    C = f_ReLU(C)
    S = f_pool(C)
    f = S.flatten()
    y_hat = f_sigmoid(f,w,bf)
    return C,f,y_hat

In [10]:
def f_getGradient_w(y_hat,y,f):
    Dw = np.squeeze(np.zeros((1,len(f))))
    a = (y_hat-y)*y_hat*(1-y_hat)
    for i in range(len(f)):
        Dw[i] = a*f[i]
    return Dw

In [12]:
def f_getGradient_f(y_hat,y,w):
    Df = np.squeeze(np.zeros((1,len(w))))
    a = (y_hat-y)*y_hat*(1-y_hat)
    for i in range(len(w)):
        Df[i] = a*w[i]
    return Df

In [13]:
def f_getGradient_bf(y_hat,y):
    Dbf = (y_hat-y)*y_hat*(1-y_hat)
    return Dbf

In [14]:
def f_getGradient_S(Df):
    n = int(len(Df)**0.5)
    DS = Df.reshape((n,n))
    return DS

In [15]:
def f_getGradient_C(DS,C):
    r = C.shape[0]
    c = C.shape[1]
    DC = np.zeros((r,c))
    for i in range(0,r,2):
        for j in range(0,c,2):
            C_block = C[i:i+2,j:j+2]
            ind = np.unravel_index(np.argmax(C_block,axis=None),C_block.shape)
            DC[i+ind[0],j+ind[1]] = DS[int(i/2),int(j/2)]
    return DC     

In [16]:
def f_getChainRuleGradients(C,DC,I,u,v):
    DKuv = 0
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            if C[i,j]>0 and i-u>=0 and j-v>=0 and i-u<C.shape[0] and j-v<C.shape[1]:
                DKuv = DKuv + (I[i-u,j-v]*DC[i,j])
    return DKuv

In [17]:
def f_getGradient_K(C,I,y_hat,y,w):
    Df = f_getGradient_f(y_hat,y,w)
    DS = f_getGradient_S(Df)
    DC = f_getGradient_C(DS,C)
    DK = np.zeros((5,5))
    for u in range(-2,3):
        for v in range(-2,3):
            DK[u+2,v+2] = f_getChainRuleGradients(C,DC,I,u,v)
    return DK,DC

In [18]:
def f_getGradient_b(C,DC):
    Db = DC[C>0].sum()
    return Db

In [19]:
def f_backwardPass(I,C,f,w,y_hat,y):
    Dw = f_getGradient_w(y_hat,y,f)
    Dbf = f_getGradient_bf(y_hat,y)
    DK,DC = f_getGradient_K(C,I,y_hat,y,w)
    Db = f_getGradient_b(C,DC)
    return DK,Db,Dw,Dbf

In [20]:
def f_initParams():
    K = 0.01*np.random.randn(5,5)
    b = 0.01*np.random.randn()
    w = np.squeeze(0.01*np.random.randn(1,256))
    bf = 0.01*np.random.randn()
    return K,b,w,bf

In [38]:
I = np.random.randint(1,255,(32,32))
y = 0
K,b,w,bf = f_initParams()
for i in range(50):
    C,f,y_hat = f_forwardPass(I,K,b,w,bf)
    print(y_hat)
    DK,Db,Dw,Dbf = f_backwardPass(I,C,f,w,y_hat,y)
    alpha = 0.001
    K = K - alpha*DK
    b = b - alpha*Db
    w = w - alpha*Dw
    bf = bf - alpha*Dbf
    


0.4920229785037398
0.243209155495617
0.15288964553136547
0.11512223380183137
0.09087306899243046
0.07531031211479094
0.06470764726937782
0.05729752408381704
0.051654478064740786
0.04719946859083668
0.04359662849889232
0.04056794188721934
0.03802546596762972
0.03587733059268647
0.03405203009207826
0.03244757526677582
0.031024403919923484
0.029752018091007512
0.028606511851509826
0.027568868555206745
0.026623761696283544
0.025758693538737613
0.02496336530069008
0.02422920923141936
0.023549035905569685
0.022915914406668098
0.022323590582331607
0.021769937027563663
0.021251060744915732
0.0207635869665655
0.020304574630617984
0.019871447996215756
0.01946194089732216
0.019074050978987237
0.018706001881194704
0.018355699463212194
0.018021189144092755
0.01770243160950251
0.017398265894446933
0.01710764556369455
0.0168296247186427
0.016563346025295723
0.016308030428783798
0.01606296828194199
0.015827511664852217
0.01560106771175928
0.015383092793555568
0.015173087429734881
0.014970591824611076


In [33]:
y_hat

0.895626736610168

In [24]:
len(f)

256

In [25]:
y_hat

0.4961399908819838

In [27]:
DK

array([[-2.97547376, -2.61208367, -1.25452159, -0.10679289, -1.12689902],
       [-2.60374285, -2.91080737, -2.63396244, -3.35799435, -3.24738384],
       [-3.56845218, -3.09257297, -1.02921786, -4.64212009, -2.74420624],
       [-3.32659253, -2.67697302, -3.12026933, -3.79688228, -1.79560188],
       [-2.45224328, -3.83832392, -1.55289401, -2.82055551, -2.72611557]])

In [28]:
Dw

array([-0.34803887, -0.81728514, -0.87373002, -1.14537418, -0.59310609,
       -0.20351208, -0.72914047, -0.88524479, -0.41872099, -0.41498201,
       -0.37638551, -0.73271683, -0.30267096, -0.51834231, -0.77523831,
       -0.30602767, -0.47541978, -0.5287802 , -0.39389314, -0.2601277 ,
       -1.15820767, -0.16973692, -0.11222836, -0.        , -0.46965639,
       -0.75465536, -0.47732463, -0.34511674, -0.30446064, -0.49239171,
       -0.40545826, -0.71765816, -0.23753064, -0.23499351, -1.42790353,
       -0.17978169, -0.51620326, -1.13837197, -0.25103392, -0.66151841,
       -1.21015924, -0.46574599, -0.43922686, -0.37903103, -0.18672564,
       -0.43357395, -1.08880907, -0.        , -0.6861942 , -0.58266859,
       -0.49492594, -0.40179354, -0.52538117, -0.50736274, -1.15901314,
       -0.39998735, -0.22748738, -0.18657371, -0.68763762, -0.        ,
       -0.54415166, -0.26445015, -0.66209606, -0.31885034, -0.18546286,
       -0.31396223, -0.89413688, -0.55533751, -0.40425145, -0.93

In [29]:
Db

-0.020351432784903194

In [30]:
Dbf

-0.1060162056883096